# 04. Widgets & Visualizations

This module shows how to work with Jupyter Widgets with anywidget. It also shows how to create a Vitessce visualization of Portal data.

## Introduction to anywidgets

[Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/) are interactive browser controls for notebooks. To solve the issue of multiple platforms with multiple ways of discovering front-end code and loading and executing JavaScript, [anywidget](https://anywidget.dev) was created.

In [ ]:
# !pip install traitlets anywidget vitessce[all]

In [ ]:
import traitlets
import anywidget

In an anywidget, JavaScript can be added with the __esm_. Let's create a simple button.

In [ ]:
class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let btn = document.createElement("button");
      btn.innerHTML = `count is ?`;
      el.appendChild(btn);
    }
    export default { render };
    """

counter = CounterWidget()
counter

This creates a button, but it does not yet have any functionality. We can create a value with a traitlet, which we can then refer to. If we add a click event listener, we can then increase the value of the button when we click on it. 

In [ ]:
# this example comes from https://anywidget.dev
class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let count = () => model.get("value");
      let btn = document.createElement("button");
      btn.innerHTML = `count is ${count()}`;
      btn.addEventListener("click", () => {
        model.set("value", count() + 1);
        model.save_changes();
      });
      model.on("change:value", () => {
        btn.innerHTML = `count is ${count()}`;
      });
      el.appendChild(btn);
    }
    export default { render };
    """
    value = traitlets.Int(0).tag(sync=True)

counter = CounterWidget()
counter

We can then get the actual value of the button. Run the cell below. Then click on the button a few more times, and run the cell below again. See how it updates?

In [ ]:
counter.value

## Introduction to Vitessce

We can use these widgets to embed interactive visualizations. [Vitessce](http://vitessce.io) is a tool for exploration of spatial single cell data.